<a href="https://colab.research.google.com/github/Yasertb/Google-Earth-Engine/blob/main/TempartureComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap as gee

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-tahmasebiyaser')

In [ ]:
Map = gee.Map(basemap= 'SATELLITE')
Map

In [ ]:
# Define the urban location of interest as a point near Ahvaz, France.
u_lon = 48.68
u_lat = 31.34
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
r_lon = 48.86
r_lat = 31.48
r_poi = ee.Geometry.Point(r_lon, r_lat)

In [ ]:
Map.addLayer(u_poi, {'color': 'blue'}, 'Urban location')
Map.addLayer(r_poi, {'color': 'orange'}, 'Rural location')

In [ ]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

In [ ]:
# Initial date of interest (inclusive).
i_date = '2020-01-01'

# Final date of interest (exclusive).
f_date = '2023-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [ ]:
scale = 1000;
elv_urban_point  = elv.sample(u_poi,scale).first().get('elevation').getInfo()
elv_urban_point

In [ ]:
lst_urban_point = lst.first().sample(u_poi,scale).first().get('LST_Day_1km').getInfo()
round((lst_urban_point*0.02)-273.15)

In [ ]:
lc_urban_point = lc.first().sample(u_poi,scale).first().get('LC_Type1').getInfo()
lc_urban_point

In [ ]:
# Get the data for the pixel intersecting the point in urban area.
lst_u_poi = lst.getRegion(u_poi, scale).getInfo()

# Get the data for the pixel intersecting the point in rural area.
lst_r_poi = lst.getRegion(r_poi, scale).getInfo()

# Preview the result.
lst_u_poi[:5]

In [ ]:
import pandas as pd

In [ ]:
def ee_array_to_df(arr, list_of_bands):

    df = pd.DataFrame(arr)
    header = df.iloc[0]
    df= df[1:]
    df.columns = header
    df = df[['longitude', 'latitude', 'time', list_of_bands]].dropna()

    for band in list_of_bands:
      df[list_of_bands]=pd.to_numeric(df[list_of_bands],errors='coerce')

    df['datetime'] = pd.to_datetime(df['time'],unit='ms')
    df = df[['time','datetime',  list_of_bands]]
    return df

In [ ]:
lst_df_urban = ee_array_to_df(lst_u_poi,'LST_Day_1km')

In [ ]:
lst_df_urban

In [ ]:
def t_modis_to_celsius(t_modis):
  t_celsius =  0.02*t_modis-273.15
  return t_celsius

In [ ]:
lst_df_urban['LST_Day_1km'] = lst_df_urban['LST_Day_1km'].apply(t_modis_to_celsius)

lst_df_rural = ee_array_to_df(lst_r_poi,'LST_Day_1km')
lst_df_rural['LST_Day_1km'] = lst_df_rural['LST_Day_1km'].apply(t_modis_to_celsius)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

In [ ]:
x_data_u = np.asanyarray(lst_df_urban['time'].apply(float))
x_data_r = np.asanyarray(lst_df_rural['time'].apply(float))

In [ ]:
y_data_u = np.asanyarray(lst_df_urban['LST_Day_1km'].apply(float))
y_data_r = np.asanyarray(lst_df_rural['LST_Day_1km'].apply(float))

In [ ]:
def fit_func(t,lst0,delta_lst,tau,phi):
  return lst0+(delta_lst/2)*np.sin(2*np.pi*t/tau+phi)

In [ ]:
## Optimize the parameters using a good start p0.
lst0 = 30
delta_lst = 50
tau = 365*24*3600*1000   # milliseconds in a year
phi = 2*np.pi*4*30.5*3600*1000/tau  # offset regarding when we expect LST(t)=LST0

In [ ]:
params_u,cov_u = optimize.curve_fit(fit_func,xdata=x_data_u,ydata=y_data_u,p0=[lst0, delta_lst, tau, phi])
params_r,cov_r = optimize.curve_fit(fit_func,xdata=x_data_r,ydata=y_data_r,p0=[lst0, delta_lst, tau, phi])

In [ ]:
print('Params urban curve:',params_u)
print('Covariance urban curve:',cov_u)
print('Params rural curve:',params_r)
print('Covariance rural curve:',cov_r)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.scatter(lst_df_urban['datetime'],lst_df_urban['LST_Day_1km'],c='orange',alpha=0.2,label='Urban Data')
ax.scatter(lst_df_rural['datetime'],lst_df_rural['LST_Day_1km'],c='Green',alpha=0.3,label='Green Area Data')
ax.plot(lst_df_urban['datetime'],fit_func(x_data_u,params_u[0],params_u[1],params_u[2],params_u[3]),label='Urban fitted curve',c="orange",lw=2.5)
ax.plot(lst_df_rural['datetime'],fit_func(x_data_r,params_r[0],params_r[1],params_r[2],params_r[3]),label='Green Area fitted curve',c="Green",lw=2.5)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Temperature [C]', fontsize=14)
ax.set_ylim(-0, 60)
ax.grid(lw=0.2)
ax.legend(fontsize=10, loc='lower right')

plt.show()